<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utiliziarán los siguientes poemas de Ursula K. Le Guin como corpus:

-Leaves
-Six Quatrains
-To the train
-Bats
-Looking Back
-How It Seems To Me
-McCoy Creek: Cattle
-Kinship
-Theory of anging
-The way
-Hymn to time
-The maenads
-Read at the award dinner
-Anonyma
-Up in a cottonwood
-On the second hill
-Desire and fear
-Coast range highway, November
-The salt
-Intimation
-Every land
-A Cannon beach crow album
-Learning the name
-Crows

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset poemas de Ursula K. Le Guin.

In [3]:
import os
import platform
os.listdir("./Data/")

['clothing_ecommerce_reviews.csv', 'nooscope.txt', 'ursulakleguin.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar los versos
df = pd.read_csv('Data/ursulakleguin.txt', sep='/n', header=None, engine='python')
df.head()

,0
0,Years do odd things to identity.
1,What does it mean to say
2,I am that child in the photograph
3,at Kishamish in 1935?
4,Might as well say I am the shadow


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 368


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[['years', 'do', 'odd', 'things', 'to', 'identity'],
 ['what', 'does', 'it', 'mean', 'to', 'say']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=2,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=40,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 368


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv))

Cantidad de words distintas en el corpus: 274


### 3 - Entrenar el modelo generador

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 20300.837890625
Loss after epoch 1: 19977.306640625
Loss after epoch 2: 16282.60546875
Loss after epoch 3: 10511.96875
Loss after epoch 4: 7019.390625
Loss after epoch 5: 6354.4921875
Loss after epoch 6: 5938.5546875
Loss after epoch 7: 5971.5703125
Loss after epoch 8: 5711.125
Loss after epoch 9: 5494.7421875
Loss after epoch 10: 5524.1953125
Loss after epoch 11: 5504.203125
Loss after epoch 12: 5616.1484375
Loss after epoch 13: 5500.6328125
Loss after epoch 14: 5751.1171875
Loss after epoch 15: 5512.640625
Loss after epoch 16: 5766.3125
Loss after epoch 17: 5634.875
Loss after epoch 18: 5824.078125
Loss after epoch 19: 5478.78125
Loss after epoch 20: 5581.734375
Loss after epoch 21: 5479.8125
Loss after epoch 22: 5616.265625
Loss after epoch 23: 5839.59375
Loss after epoch 24: 5801.296875
Loss after epoch 25: 5562.359375
Loss after epoch 26: 5570.4375
Loss after epoch 27: 5689.359375
Loss after epoch 28: 5706.703125
Loss after epoch 29: 5452.1875
Loss after epoch 

(173648, 406200)

### 4 - Ensayar

Podemos buscar la cercanía de ciertas palabras relacionadas con la naturaleza, ya que Le Guin da gran importancia al conocimiento de la misma y como dicho conocimiento abre paso a diferentes culturas (para mantener viva la discusión sobre naturaleza vs cultura tan vigente en la Antropología clásica).

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["trees"], topn=10)

[('through', 0.8789553642272949),
 ('slowly', 0.7342729568481445),
 ('old', 0.71395343542099),
 ('woman', 0.7060281038284302),
 ('forest', 0.6790982484817505),
 ('twilight', 0.676645815372467),
 ('wings', 0.6630838513374329),
 ('age', 0.6564452052116394),
 ('autumn', 0.6529855728149414),
 ('falling', 0.6472907662391663)]

In [27]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(positive=["land"], topn=10)

[('moment', 0.9716211557388306),
 ('holy', 0.9330572485923767),
 ('every', 0.912818968296051),
 ('radiance', 0.7349435091018677),
 ('is', 0.7310202717781067),
 ('dead', 0.7121484279632568),
 ('there', 0.6870577335357666),
 ('shining', 0.6785123348236084),
 ('dust', 0.6569544076919556),
 ('bear', 0.65621417760849)]

In [78]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dark"], topn=10)

[('bird', 0.8252364993095398),
 ('going', 0.6510608196258545),
 ('light', 0.6440236568450928),
 ('sit', 0.6319989562034607),
 ('home', 0.6093012094497681),
 ('autumn', 0.5976971983909607),
 ('wind', 0.5916298627853394),
 ('rimrock', 0.5910196900367737),
 ('sand', 0.5899736285209656),
 ('grey', 0.5895016193389893)]

In [79]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["still"], topn=5)

[('home', 0.8101037740707397),
 ('happy', 0.8064088821411133),
 ('go', 0.7251759767532349),
 ('down', 0.6939199566841125),
 ('woman', 0.6580080986022949)]

In [80]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["night"])

[('in', 0.08286332339048386),
 ('that', 0.00042860390385612845),
 ('at', -0.022469859570264816),
 ('shadows', -0.06383362412452698),
 ('leaves', -0.06749285012483597),
 ('to', -0.0987805724143982),
 ('wings', -0.10029339045286179),
 ('salt', -0.12174148857593536),
 ('flying', -0.12252240628004074),
 ('flickering', -0.12752129137516022)]

- Es interesante como aparecen palabras como "old", "woman" o "dream" cercanas a "tree" dado que en muchos de estos poemas Le Guin habla de si misma en una relación muy estrecha con la naturaleza, como podemos ver en el poema "leaves": 

"Might as well say I am the shadow, of a leaf of the acacia tree" 

- También podemos observar que en ocaciones parece contradictorio cuando vemos las palabras cercanas a "dark" y las más lejanas a "night" que en teoría podrían ser dos palabras cercanas entre ellas, sin embargo vemos como "wings" es lejana para "night" pero "bird" es cercana a "dark". Esto podría darse debido al uso de figuras literarias como la metáfora o el símil que alteran la relación semántica entre palabras. Por ejemplo "Day and night are much the same", o "first wave of the dark tide coming in".


- Hay algo muy conmovedor cuando se analizan las apalbras "land" y "still", porque en toda la literatura de Ursula, la tierra y la permanencia en el hogar son lugares muy importantes en la creación de sus mundos de ciencia ficción, eso podemos verlo también en sus poemas. Ver las palabras cercanas como "dead", "home", "happy" o "shining" realmente dan cuenta de la importancia del hogar en su literatura. 

"Remember me before I was a heap of salt, the laughing child who seldom did as she was told or came when she was called, the merry girl who became Lot’s bride, the happy woman who loved her wicked city. Do not remember me with pity."

### 5 - Visualizar agrupación de vectores

In [22]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [28]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

D:\FACULTAD\ESPECIALIZACIÓN\4. CUARTO BIMESTRE\NLP\env\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

D:\FACULTAD\ESPECIALIZACIÓN\4. CUARTO BIMESTRE\NLP\env\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



<img src="./Resultados tp_3/1.png">
<img src="./Resultados tp_3/2.png">
<img src="./Resultados tp_3/3.png">
<img src="./Resultados tp_3/4.png">
<img src="./Resultados tp_3/5.png">
<img src="./Resultados tp_3/.png">
<img src="./Resultados tp_3/1.png">